In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier, plot_importance
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import seaborn as sns

C:\Users\mingt\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Split the file since its too big for github, extract the file outside of .git repository
"""
hpg_res = pd.read_csv('C:\\Users\\mingt\\Downloads\\hpg_reserve.csv\\hpg_reserve.csv')
hpg_res.info()
df1 = hpg_res.iloc[0:100160]
df2 = hpg_res.iloc[100161:200320]
df1.to_csv('hpg_reserve1.csv')
df2.to_csv('hpg_reserve2.csv')
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB


In [2]:
air_reserve = pd.read_csv('air_reserve.csv')
air_store_info= pd.read_csv('air_store_info.csv')
air_visit_data = pd.read_csv('air_visit_data.csv')
date_info = pd.read_csv('date_info.csv')
hpg_store_info = pd.read_csv('hpg_store_info.csv')
store_id_relation = pd.read_csv('store_id_relation.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
#concat files back after its been uploaded on github
hpg_reserve1 = pd.read_csv('hpg_reserve1.csv')
hpg_reserve2 = pd.read_csv('hpg_reserve2.csv')

hpg_reserve = pd.concat([hpg_reserve1, hpg_reserve2])
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])

In [4]:
air_combined = pd.merge(air_reserve, air_store_info, on='air_store_id', how='outer')
hpg_combined = pd.merge(hpg_reserve, hpg_store_info, on='hpg_store_id', how='left')

df = store_id_relation.merge(hpg_combined, on='hpg_store_id', how='left')
df2 = air_combined.merge(df, on='air_store_id', how='left')

### Primary key on column: 'air_store_id'

In [5]:
master_col = df2[['air_store_id']]
df2.head(5)

,air_store_id,visit_datetime_x,reserve_datetime_x,reserve_visitors_x,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_store_id,Unnamed: 0,visit_datetime_y,reserve_datetime_y,reserve_visitors_y,hpg_genre_name,hpg_area_name,latitude_y,longitude_y
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
2,air_877f79706adbfb06,2016-01-02 18:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3,air_877f79706adbfb06,2016-01-02 21:00:00,2016-01-01 16:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
4,air_877f79706adbfb06,2016-01-08 21:00:00,2016-01-04 20:00:00,2.0,Japanese food,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN


#### 11171047 entries in 'air_store_id', with 829 unique restaurants. 

In [6]:
print(df2.info())
print(len(df2['air_store_id'].value_counts()))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171047 entries, 0 to 1171046
Data columns (total 17 columns):
air_store_id          1171047 non-null object
visit_datetime_x      1170265 non-null object
reserve_datetime_x    1170265 non-null object
reserve_visitors_x    1170265 non-null float64
air_genre_name        1171047 non-null object
air_area_name         1171047 non-null object
latitude_x            1171047 non-null float64
longitude_x           1171047 non-null float64
hpg_store_id          1114437 non-null object
Unnamed: 0            1112519 non-null float64
visit_datetime_y      1112519 non-null datetime64[ns]
reserve_datetime_y    1112519 non-null datetime64[ns]
reserve_visitors_y    1112519 non-null float64
hpg_genre_name        451960 non-null object
hpg_area_name         451960 non-null object
latitude_y            451960 non-null float64
longitude_y           451960 non-null float64
dtypes: datetime64[ns](2), float64(7), object(8)
memory usage: 160.8+ MB
None
829


### Add features to date_info

In [6]:
date_info['calendar_date'] = pd.to_datetime(date_info['calendar_date']).dt.date.astype(str)
#date_info['holiday_flg'] = date_info['holiday_flg'].map({1: 'Yes', 0: 'No'})
date_info['MTWTF'] = date_info['day_of_week'].map({'Monday': 1, 
                                                   'Tuesday': 2, 
                                                   'Wednesday': 3, 
                                                   'Thursday': 4, 
                                                   'Friday': 5,
                                                   'Saturday': 6, 
                                                   'Sunday': 7})

date_info['weekend_or_weekday'] = date_info['day_of_week'].map({'Monday': 0, 
                                                                'Tuesday': 0, 
                                                                'Wednesday': 0, 
                                                                'Thursday': 0, 
                                                                'Friday': 0,
                                                                'Saturday': 1, 
                                                                'Sunday': 1})


date_info2 = date_info.drop(['day_of_week'], axis=1)
date_info2.head(5)

,calendar_date,holiday_flg,MTWTF,weekend_or_weekday
0,2016-01-01,1,5,0
1,2016-01-02,1,6,1
2,2016-01-03,1,7,1
3,2016-01-04,0,1,0
4,2016-01-05,0,2,0


### Join sample_submission to date_info

In [8]:
sub_store = sample_submission['id'].apply(lambda x: str(x).split('_', 2)[:2])
sub_dates = pd.to_datetime(sample_submission['id'].apply(lambda x: str(x).split('_', 2)[2]).rename('Date'))
sub_stores = pd.Series(['_'.join(x) for x in sub_store]).rename('air_store_id')

sub_dt = pd.DataFrame({
        'air_store_id': sub_stores,
        'date': sub_dates.dt.date.astype(str),
        #'year': sub_dates.dt.year,
        'month': sub_dates.dt.month,
        'day': sub_dates.dt.day })

sub_df = pd.concat([sample_submission, sub_dt], axis=1)
sub_df2 = sub_df.merge(date_info2, left_on= sub_df['date'], right_on=date_info['calendar_date'])
sub_df3 = sub_df2.drop(['id', 'calendar_date', 'date'], axis =1)
sub_df3.head(5)

,visitors,air_store_id,day,month,holiday_flg,MTWTF,weekend_or_weekday
0,0,air_00a91d42b08b08d9,23,4,0,7,1
1,0,air_0164b9927d20bcc3,23,4,0,7,1
2,0,air_0241aa3964b7f861,23,4,0,7,1
3,0,air_0328696196e46f18,23,4,0,7,1
4,0,air_034a3d5b40d5b1b1,23,4,0,7,1


### Split datetime columns, and join with date_info

In [9]:
a = pd.to_datetime(df2['visit_datetime_x'])
#b = pd.to_datetime(df2['reserve_datetime_x'])
#c = pd.to_datetime(df2['visit_datetime_y'])
#d = pd.to_datetime(df2['reserve_datetime_y'])

datetime_df =pd.DataFrame({
        'air_visit_date': a.dt.date.astype(str),
        #'year': a.dt.year,
        'month': a.dt.month,
        'day': a.dt.day,
        
        #'air_visit_hour': a.dt.hour,
        #'air_visit_date': a.dt.date,
        #'air_visit_year': b.dt.year,
        
        #'air_reserve_month': b.dt.month,
        #'air_reserve_day': b.dt.day,
        #'air_reserve_hour': b.dt.hour,
         
        #'hpg_visit_date': c.dt.date,
        #'hpg_visit_year': c.dt.year,
        #'hpg_visit_month': c.dt.month,
        #'hpg_visit_day': c.dt.day,
        #'hpg_visit_hour': c.dt.hour,
        
        #'hpg_reserve_date': d.dt.date,
        #'hpg_reserve_year': d.dt.year,
        #'hpg_reserve_month': d.dt.month,
        #'hpg_reserve_day': d.dt.day,
        #'hpg_reserve_hour': d.dt.hour        
    }).fillna(0)

datetime_df2 = datetime_df.merge(date_info, left_on=datetime_df['air_visit_date'], 
                                 right_on=date_info['calendar_date'])

### Encode all categorical variable, then join with submission_file and training set

In [10]:
categorical_df = pd.concat([master_col, df2[['air_genre_name','air_area_name','latitude_x','longitude_x',
                      'hpg_genre_name','hpg_area_name']].fillna('None_Stated').apply(LabelEncoder().fit_transform)], axis=1)

#categorical_df_onehot = pd.concat([master_col, df2[['air_genre_name','air_area_name','latitude_x','longitude_x',
                      #'hpg_genre_name','hpg_area_name']].fillna('None_Stated').apply(OneHotEncoder().fit_transform)], axis=1)


categorical_dummy = pd.get_dummies(df2[['air_genre_name','air_area_name','latitude_x','longitude_x', 'hpg_genre_name','hpg_area_name']].fillna('None_Stated'))

sub_df4 = sub_df3.merge(categorical_df, left_on=sub_df3['air_store_id'], right_on=categorical_df['air_store_id'])  

combined_visitors = pd.Series(df2['reserve_visitors_x'].fillna(0) + df2['reserve_visitors_y'].fillna(0)).rename('visitors')


train_df = pd.concat([datetime_df2, combined_visitors, categorical_df], axis=1)

### Encode all air_store_id together, drop unneeded columns

In [14]:
lbl= LabelEncoder()
ids = lbl.fit_transform(train_df['air_store_id'].append(sub_df4['air_store_id_x']))
train_df['air_store_id_num'] = pd.Series(ids[:1171046])
sub_df4['air_store_id_num'] = pd.Series(ids[1171047:])

train_df2 = train_df.drop(['air_store_id', 'air_visit_date', 'calendar_date', 'day_of_week'], axis=1)
sub_df5 = sub_df4.drop(['air_store_id_x', 'air_store_id_y'], axis=1).reindex(columns=list(train_df2.columns.values))

In [18]:
combined_visitors.isnull().values.any()

False

In [15]:
train_df2

,day,month,holiday_flg,MTWTF,weekend_or_weekday,visitors,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_genre_name,hpg_area_name,air_store_id_num
0,1.0,1.0,1.0,5.0,0.0,1.0,8,62,54,83,10,16,426.0
1,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
2,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
3,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
4,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
5,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
6,1.0,1.0,1.0,5.0,0.0,3.0,8,62,54,83,10,16,426.0
7,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0
8,1.0,1.0,1.0,5.0,0.0,4.0,8,62,54,83,10,16,426.0
9,1.0,1.0,1.0,5.0,0.0,2.0,8,62,54,83,10,16,426.0


In [26]:
sub_df5.head(5)

,day,month,holiday_flg,MTWTF,weekend_or_weekday,visitors,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_genre_name,hpg_area_name,air_store_id_num
0,23,4,0,7,1,0,6,44,68,85,10,16,0
1,23,4,0,7,1,0,6,44,68,85,10,16,0
2,23,4,0,7,1,0,6,44,68,85,10,16,0
3,23,4,0,7,1,0,6,44,68,85,10,16,0
4,23,4,0,7,1,0,6,44,68,85,10,16,0


### Make some descriptive visualizations

In [11]:
vis_store = train_df2['visitors'].groupby(train_df2['air_store_id_num']).describe()
vis_dow = train_df2['visitors'].groupby(train_df2['MTWTF']).describe()
vis_store_dow = train_df2['visitors'].groupby([train_df2['air_store_id_num'], train_df2['MTWTF']]).describe()

In [67]:
unique_stores =train_df2['air_store_id_num'].unique()

vis_store_min = train_df2.groupby(['air_store_id_num','MTWTF'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
vis_store_mean = train_df2.groupby(['air_store_id_num','MTWTF'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
vis_store_med = train_df2.groupby(['air_store_id_num','MTWTF'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
vis_store_max = train_df2.groupby(['air_store_id_num','MTWTF'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
vis_store_count = train_df2.groupby(['air_store_id_num','MTWTF'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_visitors'})


all_stores =train_df2['air_store_id_num'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id_num': unique_stores, 'MTWTF': [i]*len(unique_stores)}) for i in range(1,8)], axis=0, ignore_index=True).reset_index(drop=True)

In [68]:

stores = pd.merge(stores, vis_store_min, how='left', on=['air_store_id_num','MTWTF'])

stores = pd.merge(stores, vis_store_mean, how='left', on=['air_store_id_num','MTWTF']) 

stores = pd.merge(stores, vis_store_med, how='left', on=['air_store_id_num','MTWTF']) 

stores = pd.merge(stores, vis_store_max, how='left', on=['air_store_id_num','MTWTF'])#.fillna(0)



In [73]:
train_df3 = pd.merge(stores, train_df2, how='left', on=['air_store_id_num','MTWTF']).dropna(axis=0)

In [76]:
train_df3

,MTWTF,air_store_id_num,min_visitors,mean_visitors,median_visitors,max_visitors,day,month,holiday_flg,weekend_or_weekday,visitors,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_genre_name,hpg_area_name
2,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,6.0,4.0,18.0,106.0,105.0,13.0,7.0
3,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,4.0,4.0,18.0,106.0,105.0,13.0,7.0
4,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,4.0,4.0,18.0,106.0,105.0,13.0,7.0
5,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,4.0,4.0,18.0,106.0,105.0,13.0,7.0
6,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,5.0,4.0,18.0,106.0,105.0,13.0,7.0
7,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,10.0,4.0,18.0,106.0,105.0,13.0,7.0
8,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,12.0,4.0,18.0,106.0,105.0,13.0,7.0
9,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,4.0,4.0,18.0,106.0,105.0,13.0,7.0
10,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,4.0,4.0,18.0,106.0,105.0,13.0,7.0
11,1,712.0,4.0,14.346240,11.0,71.0,18.0,1.0,0.0,0.0,10.0,4.0,18.0,106.0,105.0,13.0,7.0


In [57]:
sub_df5

,day,month,holiday_flg,MTWTF,weekend_or_weekday,visitors,air_genre_name,air_area_name,latitude_x,longitude_x,hpg_genre_name,hpg_area_name,air_store_id_num
0,23,4,0,7,1,0,6,44,68,85,10,16,0
1,23,4,0,7,1,0,6,44,68,85,10,16,0
2,23,4,0,7,1,0,6,44,68,85,10,16,0
3,23,4,0,7,1,0,6,44,68,85,10,16,0
4,23,4,0,7,1,0,6,44,68,85,10,16,0
5,23,4,0,7,1,0,6,44,68,85,10,16,0
6,23,4,0,7,1,0,6,44,68,85,10,16,0
7,23,4,0,7,1,0,6,44,68,85,10,16,0
8,23,4,0,7,1,0,6,44,68,85,10,16,0
9,23,4,0,7,1,0,6,44,68,85,10,16,0


In [23]:
def rmsle(h, y): 
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y

    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(h + 1) - np.log(y + 1)).mean())


X_train, X_test, y_train, y_test = train_test_split(train_df2, train_df2['visitors'], test_size=0.1, random_state=7)


xgbR = XGBRegressor(learning_rate=0.1,
                    n_estimators=132,
                   objective='reg:linear')

xgbR.fit(X_train, y_train)


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=132, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [15]:
pred = xgbR.predict(X_test)
mse = rmsle(pred, y_test)
print(mse)

0.000581949284692


In [23]:
test-rmse-mean	test-rmse-std	train-rmse-mean	train-rmse-std
128	0.005455	0.002128	0.000257	0.000040
129	0.005455	0.002128	0.000244	0.000037
130	0.005455	0.002128	0.000237	0.000038
131	0.005455	0.002128	0.000225	0.000035
132	0.005454	0.002128	0.000217	0.000037

dmatrix = xgb.DMatrix(train_df2, train_df2['visitors'])

xgb_params_cv = dict(eta=0.1,
                  objective='reg:linear')

best = xgb.cv(params = xgb_params_cv, dtrain=dmatrix, num_boost_round=10000, metrics='rmse', early_stopping_rounds=1000, verbose_eval=True)

best.tail(5)     

[0]	train-rmse:8.48332+0.0150103	test-rmse:8.4834+0.0301305
[1]	train-rmse:7.63577+0.0135089	test-rmse:7.63582+0.0271652
[2]	train-rmse:6.8729+0.0121559	test-rmse:6.87291+0.0244977
[3]	train-rmse:6.18626+0.0109389	test-rmse:6.18629+0.0221324
[4]	train-rmse:5.56824+0.00985373	test-rmse:5.56829+0.0198993
[5]	train-rmse:5.01199+0.00886462	test-rmse:5.01195+0.0179895
[6]	train-rmse:4.51131+0.00798038	test-rmse:4.51127+0.0162114
[7]	train-rmse:4.06066+0.00717846	test-rmse:4.06064+0.0145825
[8]	train-rmse:3.65505+0.00646405	test-rmse:3.65502+0.0131354
[9]	train-rmse:3.28995+0.00581133	test-rmse:3.28992+0.0118407
[10]	train-rmse:2.9613+0.00523018	test-rmse:2.96137+0.0106914
[11]	train-rmse:2.66549+0.00470522	test-rmse:2.66555+0.00961939
[12]	train-rmse:2.39924+0.00423721	test-rmse:2.39926+0.00865864
[13]	train-rmse:2.15959+0.00381368	test-rmse:2.15958+0.00782618
[14]	train-rmse:1.94388+0.00342734	test-rmse:1.94391+0.00704309
[15]	train-rmse:1.74973+0.00308362	test-rmse:1.74978+0.00634334
[16]

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
128,0.005455,0.002128,0.000257,0.000040
129,0.005455,0.002128,0.000244,0.000037
130,0.005455,0.002128,0.000237,0.000038
131,0.005455,0.002128,0.000225,0.000035
132,0.005454,0.002128,0.000217,0.000037


In [36]:
param_test1 = {
 'max_depth':[5,6,7]}

param_test2 = {
'min_child_weight':[1,2,3]}

param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]}

param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]}

param_test5 = {
 'reg_alpha':[1e-5, 1e-6, 1e-7]}

grid = GridSearchCV(estimator=XGBRegressor(learning_rate=0.1,
                  n_estimators=132,
                  max_depth=5,
                  min_child_weight=1,
                  gamma=0,
                  subsample=0.9,
                  colsample_bytree=0.9,
                  #objective='reg:linear',
                  nthread=4,
                  scale_pos_weight=1,
                  reg_alpha=1e-6,
                  seed=7)
                 #param_grid = param_test5) scoring='neg_mean_squared_error')

grid.fit(X_train,y_train)
grid.grid_scores_, grid.best_params_, grid.best_score_

C:\Users\mingt\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.99998, std: 0.00000, params: {'reg_alpha': 1e-05},
  mean: 0.99998, std: 0.00000, params: {'reg_alpha': 1e-06},
  mean: 0.99998, std: 0.00000, params: {'reg_alpha': 1e-07}],
 {'reg_alpha': 1e-06},
 0.9999821901703938)

In [14]:
xgbr2 = XGBRegressor(learning_rate=0.1,
                  n_estimators=132,
                  max_depth=5,
                  min_child_weight=1,
                  gamma=0,
                  subsample=0.9,
                  colsample_bytree=0.9,
                  #objective='reg:linear',
                  #nthread=4,
                  scale_pos_weight=1,
                  reg_alpha=1e-6,
                  seed=7)

xgbr2.fit(train_df2, train_df2['visitors'])

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=132, nthread=-1,
       objective='reg:linear', reg_alpha=1e-06, reg_lambda=1,
       scale_pos_weight=1, seed=7, silent=True, subsample=0.9)

In [15]:
gbr = GradientBoostingRegressor(learning_rate=0.1,
                                  n_estimators=132,
                                  max_depth=5,
                                  subsample=0.9)
gbr.fit(train_df2, train_df2['visitors'])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [20]:
pred2 = xgbr2.predict(sub_df5)

In [28]:
final = pd.Series(pred2).rename('vis_pred')

In [30]:
submission = pd.concat([sample_submission, final], axis=1)

In [32]:
final

0           0.228044
1           0.228044
2           0.228044
3           0.228044
4           0.228044
5           0.228044
6           0.228044
7           0.228044
8           0.228044
9           0.228044
10          0.228044
11          0.228044
12          0.228044
13          0.228044
14          0.228044
15          0.228044
16          0.228044
17          0.228044
18          0.228044
19          0.228044
20          0.228044
21          0.228044
22          0.228044
23          0.228044
24          0.228044
25          0.228044
26          0.228044
27          0.228044
28          0.228044
29          0.228044
              ...   
45670491    0.185518
45670492    0.191882
45670493    0.194976
45670494    0.197557
45670495    0.190696
45670496    0.190696
45670497    0.181419
45670498    0.184472
45670499    0.181281
45670500    0.184923
45670501    0.186605
45670502    0.186605
45670503    0.186605
45670504    0.177328
45670505    0.180382
45670506    0.180382
45670507    0